<a href="https://www.kaggle.com/code/odissey114/yolov8-emotions-on-faces-multi-detection-ipynb?scriptVersionId=193070169" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !wget --quiet https://github.com/sizhky/torch_snippets.git
!pip install torch_snippets
from torch_snippets import *
from torch_snippets.inspector import inspect

In [ ]:
# from torch_snippets import *
from IPython import display
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchinfo import summary
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T, models, datasets
from xml.etree import ElementTree as et
import torchvision
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.patches as patches
import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
%matplotlib inline

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Здесь чисто переводчик:
!pip install googletrans==3.1.0a0
# import googletrans
# from pprint import pprint
# print("Number of Supported Languages:", len(googletrans.LANGUAGES))
# print(googletrans.LANGUAGES)
from googletrans import Translator
translator = Translator()

In [ ]:
# блок для Ёлы8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xml.etree.cElementTree as ET
import glob 
import os
import json
import random 
import shutil 

from PIL import Image, ImageOps

In [ ]:
# Пути до картинок и аннотаций
ims = '/kaggle/input/8-facial-experssions/images'
anns = '/kaggle/input/8-facial-experssions/labels'

In [ ]:
images = [str(i) for i in Glob('/kaggle/input/8-facial-experssions/images/*')]
rows,cols=3,3
fig=plt.figure(figsize=(10,10))
for i in range(1,rows*cols+1):
    fig.add_subplot(rows,cols,i)
    img_path=images[i]
#     msk=mask_to_mask(anns[i])
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     msk=cv2.imread(msk_path)
    plt.imshow(img)
#     plt.imshow(msk,alpha=0.3)
    plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# Смотрим картинку
show('/kaggle/input/8-facial-experssions/images/sad_085.jpg',sz=8)

In [ ]:
def create_dataset(ims_fold, anns_fold):#, targ_data):
#     assert isinstance(data_path, str) # проверка типа данных
#     assert isinstance(target_path, str) # проверка типа данных

    images = [str(i) for i in Glob('/kaggle/input/8-facial-experssions/images/*')]
    annots = [str(i) for i in Glob('/kaggle/input/8-facial-experssions/labels/*')]
    
    emotions = []
    
    dict_paths = {
        "image": [],
        "annotation": [],
        "emotion": [],
        "emotion_ru": [],
#         "em_code": []
    }

    for i in images:
        name = i.split('/')[-1][:-4]
#         if name.split('_')[0] not in emotions: emotions.append(name.split('_')[0])
#         print(name)
        translated_text = translator.translate(name.split('_')[0], src='en', dest = 'ru')
        if f'{ims_fold}/{name}.jpg' in images and f'{anns_fold}/{name}.txt' in annots:
            dict_paths["image"].append(f"{ims_fold}/{name}.jpg")
            dict_paths["annotation"].append(f"{anns_fold}/{name}.txt")
            dict_paths["emotion"].append(name.split('_')[0])
            dict_paths["emotion_ru"].append(translated_text.text)
        else: continue

    dataframe = pd.DataFrame(
        data=dict_paths,
        index=np.arange(0, len(dict_paths["image"]))
    )
#     print(emotions)
    return dataframe

In [ ]:
df = create_dataset(ims, anns)
df.head(5)
df.shape

In [ ]:
df.head(5)

In [ ]:
df.emotion.unique(), df.emotion_ru.unique()

In [ ]:
lst = list(df.emotion_ru.unique())
lst.insert(0, 'background')
label2targets = {l: t for t, l in enumerate(lst)}
targets2label = {t: l for l, t in label2targets.items()}
num_classes = len(lst)
print(lst)
print(label2targets)
print(targets2label)
print(num_classes)

In [ ]:
for i in open('/kaggle/input/8-facial-experssions/labels/contempt_030.txt', 'r'):
    print(i)

In [ ]:
f = open('/kaggle/input/8-facial-experssions/labels/contempt_030.txt', 'r')
for label in f:
    print(label)

In [ ]:
def DataSet(d_f):
    picture =[]#
    way_pic =[]#
    way_ann =[]#
    emotion = []
    emotion_ru = []
    target = []
    X_min = []#
    Y_min = []#
    X_max = []#
    Y_max = []#
    width = []#
    height = []#
    for _ in d_f.index:
        f = open(d_f.annotation.iloc[_], 'r')
        for label in f:
            class_id, x_center, y_center, wid, heig = map(float, label.split())
            pic_shape = read(d_f['image'][_], 3).shape
            h, w, d = pic_shape
            X_min.append(round((x_center - wid/2)*w))
            Y_min.append(round((y_center - heig/2)*h))
            X_max.append(round((x_center + wid/2)*w))
            Y_max.append(round((y_center + heig/2)*h))
            target.append(label2targets[d_f.emotion_ru[_]])
            way_pic.append(d_f['image'][_]) # путь картинки
            way_ann.append(d_f['annotation'][_]) # путь аннотации
            picture.append(d_f['image'][_].split('/')[-1]) # имя картинки

            pic_shape = read(d_f['image'][_]).shape
#             print(pic_shape)
            width.append(pic_shape[1])
            height.append(pic_shape[0])

    name_col = ['ImageID', 'LabelName', 'X_min', 'Y_min', 'X_max', 'Y_max','width','height', 'way_pic', 'way_ann']
    lst = picture, target, X_min, Y_min, X_max, Y_max, width, height, way_pic, way_ann
    df = pd.DataFrame(np.column_stack(lst), columns = name_col)
#     DF_RAW = df = df[df.ImageID.str.contains('.jpg')]

#     df.LabelName = np.around(df.LabelName.astype(float))
    df.LabelName = np.around(df.LabelName.astype(int))
    df.X_min = np.around(df.X_min.astype(float),6)
    df.X_max = np.around(df.X_max.astype(float),6)
    df.Y_min = np.around(df.Y_min.astype(float),6)
    df.Y_max = np.around(df.Y_max.astype(float),6)
    df.width = np.around(df.width.astype(int))
    df.height = np.around(df.height.astype(int))
    return df

In [ ]:
data = DataSet(df)
data

In [ ]:
data.dtypes

In [ ]:
data.loc[0].width

In [ ]:
def f(x):
    arr = np.array(data.iloc[x,2:6])
    width = data.width[x]
    height = data.height[x]
    print(arr, width, height)
    # arr = arr[0::2]*width
    # arr = arr[1::2]*height
    # arr
    box = []
    for i,j in enumerate(arr):
        if i%2 == 0:
            box.append(int(j))#*width))
        else: box.append(int(j))#*height))
    эмоция = data.iloc[x,1]
    print(box, эмоция)
    ss = data.way_pic[x]
    return show(ss, bbs = [box], texts=[targets2label[эмоция]], sz=10)

In [ ]:
for i in range(60,67):
    f(i)

In [ ]:
data.head(2)

In [ ]:
data.way_pic.iloc[0]

In [ ]:
np.array(data[data.way_pic == '/kaggle/input/8-facial-experssions/images/sad_097.jpg'].iloc[:,1:6])

# **4. Компатовка данных и создание файлов txt с аннотациями для подачи в елу**

In [ ]:
list(targets2label.values())[1:]

In [ ]:
# Перевод XML-формат в TXT-формат
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0])/2)/w
    y_center = ((bbox[3] + bbox[1])/2)/h
    
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    
    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center, width, height
    w_half_len = (bbox[2] * w)/2
    h_half_len = (bbox[3] * h)/2
    
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    
    return [xmin, ymin, xmax, ymax]

classes = list(targets2label.values())[1:]# ["Occupied", "Vacant"] # классы
# [1, 2, 3, 4, 5, 6, 7, 8]
# ['грустный', 'удивлен', 'отвращение', 'страх', 'нейтральный', 'презрение', 'счастливый', 'злость'] - здесь вот это

input_ims_dir = "/kaggle/input/8-facial-experssions/images"
input_anns_dir = "/kaggle/input/8-facial-experssions/labels"
output_dir = "/kaggle/working/labels"
# image_dir = "/kaggle/input/face-mask-detection/images"

if not os.path.isdir(output_dir): # создане папки для выхода
    os.mkdir(output_dir)
    
files = data.way_ann.unique() # соединение в путь, хотя можно просто было папку указать
                                                # и через всписковые включения загнать в str
len(files)

In [ ]:
# list(targets2label.values())[1:]

In [ ]:
data.head(2)

In [ ]:
data.way_pic.iloc[0]

In [ ]:
data.way_ann.iloc[0]

In [ ]:
fil = data.way_ann.iloc[0]
f'/kaggle/input/8-facial-experssions/images/{fil.split("/")[-1][:-4]}jpg'

In [ ]:
# data.way_pic.unique()

In [ ]:
# os.mkdir('/kaggle/working/labels')
for fil in files:
    basename = fil.split('/')[-1]
    filename = basename.split('.')[0]
    #---------------------------------------------------------------------
    # блок проверки: типа если существует картинка с таким же названием, то работаем, иначе пропускаем
    # os.path.exists - проверка наличия
#     print(fil.split('/')[-2:])
    if f'/kaggle/input/8-facial-experssions/images/{fil.split("/")[-1][:-4]}.jpg' not in data.way_pic.unique():
        print(f"{filename} image does not exist!")
        continue 
    #---------------------------------------------------------------------
    result = []
    width = data[data.way_ann == fil].width.unique()[0]
    height = data[data.way_ann == fil].height.unique()[0]
    for index, row in data[data.way_ann == fil].iterrows():
#     print(row['c1'], row['c2'])
        label = row['LabelName']
            # Проверяет наличие новых классов и добавляет их в список
        if targets2label[label] not in classes:
            classes.append(label)
        index = classes.index(targets2label[label]) # выводит индекс класса в списке
        pil_bbox = row.iloc[2:6].to_list() # типа достает [28, 55, 46, 71]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height) # типа преобразует в (x_центр, y_центр, w, h)
        # так выглядит выход:
#         [0.0925, 0.27876106194690264, 0.045, 0.07079646017699115]
#         [0.26125, 0.30973451327433627, 0.0325, 0.07079646017699115]
#         [0.44, 0.30973451327433627, 0.085, 0.17699115044247787]
        
        # преобразование данных в строку
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        # так выглядит выход:
#         0.0925 0.27876106194690264 0.045 0.07079646017699115
#         0.26125 0.30973451327433627 0.0325 0.07079646017699115
#         0.44 0.30973451327433627 0.085 0.17699115044247787
#         print(bbox_string)
        result.append(f"{index} {bbox_string}")
        # короче на выходе список строк с метками и коробками [x,y,w,h], разделенных пробелами
        # выводится для кадой картинки
        # ['1 0.0925 0.27876106194690264 0.045 0.07079646017699115', 
        #  '1 0.26125 0.30973451327433627 0.0325 0.07079646017699115']

#---------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------
# создаем файлы txt для каждой картинки с классами и коробками и файл txt c классами
    if result: 
        # создаем файлы txt для каждой картинки, в виде (формата yolo для каждого xml-файла):
#         1 0.625 0.3651685393258427 0.08 0.13857677902621723
#         1 0.8825 0.3089887640449438 0.075 0.12359550561797752
#         1 0.335 0.40074906367041196 0.075 0.14232209737827714
#         1 0.39 0.41198501872659177 0.05 0.0898876404494382
#         1 0.22125 0.4044943820224719 0.0575 0.1198501872659176
#         1 0.04125 0.46629213483146065 0.0725 0.12359550561797752
        with open(os.path.join(output_dir, f"{filename}.txt"), # создаем текстовый файл
                 "w", encoding = 'utf-8') as f:
            f.write("\n".join(result)) # записываем в файл данные, каждый в новой строке
            #_____________________________
#             print(f)

# создает файл txt с классами, здесь файл со списком: ["without_mask", "with_mask", "mask_weared_incorrect"]
with open("/kaggle/working/classes.txt", "w", encoding = 'utf-8') as f:
    f.write(json.dumps(classes)) # записывает файл типа json
    
    
    
    
#     print(width, height)

In [ ]:
# # Текстовая версия приведенной выше метки в формате XML
with open('/kaggle/working/labels/disgust_035.txt') as f:
    contents = f.read()
    print(contents)

In [ ]:
# Название и создание файлов, подготовленных для Yolov8
# ЗДЕСЬ СОЗДАЕМ НЕОБХОДИМЫЕ ПАПКИ
if os.path.isfile('/kaggle/working/data'):
    print('There is  folder !')
else:
    os.mkdir('/kaggle/working/data')
os.mkdir('/kaggle/working/data/train')
os.mkdir('/kaggle/working/data/val')
os.mkdir('/kaggle/working/data/test')
os.mkdir('/kaggle/working/data/train/images')
os.mkdir('/kaggle/working/data/train/labels')
os.mkdir('/kaggle/working/data/test/images')
os.mkdir('/kaggle/working/data/test/labels')
os.mkdir('/kaggle/working/data/val/images')
os.mkdir('/kaggle/working/data/val/labels')

In [ ]:
# Создаем список названий картинок без расширений
metarial = []

for i in data.way_pic.unique():
    srt = i[:-4].split('/')[-1]
    metarial.append(srt)
print(metarial)
print(len(metarial))

In [ ]:
# Короче распределяем картинки по папкам и файлы txt с аннотациями, коробки всякие
# для подачи в елу8.


def preparedata(main_txt_file, main_img_file, train_size, test_size, val_size):
    for i in range(0, train_size):
        source_txt = main_txt_file + "/" + metarial[i] + ".txt"
        source_img = main_img_file + "/" + metarial[i] + ".jpg"
        
        mstring = metarial[i]
        train_destination_txt = "/kaggle/working/data/train/labels" + "/" + metarial[i] + ".txt"
        train_destination_png = "/kaggle/working/data/train/images" + "/" + metarial[i] + ".jpg"
        
        shutil.copy(source_txt, train_destination_txt)
        shutil.copy(source_img, train_destination_png)
    
    for l in range(train_size, train_size + test_size):
        source_txt = main_txt_file + "/" + metarial[l] + ".txt"
        source_img = main_img_file + "/" + metarial[l] + ".jpg"
        
        mstring = metarial[l]
        test_destination_txt = "/kaggle/working/data/test/labels" + "/" + metarial[l] + ".txt"
        test_destination_png = "/kaggle/working/data/test/images" + "/" + metarial[l] + ".jpg"
        
        shutil.copy(source_txt, test_destination_txt)
        shutil.copy(source_txt, test_destination_png)
        
    
    for n in range(train_size + test_size, train_size + test_size + val_size):
        source_txt = main_txt_file + "/" + metarial[n] + ".txt"
        source_img = main_img_file + "/" + metarial[n] + ".jpg"
        
        mstring = metarial[n]
        val_destination_txt = "/kaggle/working/data/val/labels" + "/" + metarial[n] + ".txt"
        val_destination_png = "/kaggle/working/data/val/images" + "/" + metarial[n] + ".jpg"
        
        shutil.copy(source_txt, val_destination_txt)
        shutil.copy(source_img, val_destination_png)

In [ ]:
preparedata("/kaggle/working/labels", '/kaggle/input/8-facial-experssions/images', 420, 100, 55)

In [ ]:
# создали что-то типа текскового файла с данными yaml_text, прям и форма такая-же и больше ничего
yaml_text = """train: /kaggle/working/data/train/images
val: /kaggle/working/data/val/images

nc: 8
names: ['грустный', 'удивлен', 'отвращение', 'страх', 'нейтральный', 'презрение', 'счастливый', 'злость']"""

with open("/kaggle/working/data/data.yaml", 'w') as file:
    file.write(yaml_text)

In [ ]:
%cat /kaggle/working/data/data.yaml # типа что-то типа содержимого посмотрели штоли %cat што за такое, в инете нет

# **6. Грузим модель елу8**

In [ ]:
# мы устанавливаем библиотеку ultralytics, чтобы использовать библиотеку YoloV8
%pip install ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

# **7. Обучение** типа

In [ ]:
# типа папку с результатами создаем
if os.path.isfile('/kaggle/working/Result/'):
    print("There is result folder")
else:
    os.mkdir("/kaggle/working/Result/")

In [ ]:
%cd /kaggle/working/Result/

In [ ]:
!yolo train model=yolov8n.pt data = /kaggle/working/data/data.yaml epochs = 80 imgsz = 640

In [ ]:
Image.open("/kaggle/working/Result/runs/detect/train/results.png")

# **Проба пера.**

In [ ]:
# Процесс обнаружения, который мы выполнили на основе тестового набора данных
!yolo predict model = /kaggle/working/Result/runs/detect/train/weights/best.pt  source='/kaggle/working/data/val/images'

In [ ]:
for i in Glob('/kaggle/working/Result/runs/detect/predict/'):
    print(i)
    show(i)